# 🧪 Panel Chemistry - PDBe Molstar

The Panel Chemistry `PDBeMolStar` allows you to use the powerful PBDe JS plugin implementation of the Mol* ('MolStar') viewer. 

[PDBe Mol*](https://github.com/PDBeurope/pdbe-molstar) using Python 🐍 and [HoloViz Panel](https://panel.holoviz.org/) ❤️. 

![PDBe MolStarWebComponent](../assets/PDBe_Molstar.png) (needs updating)

When using Mol*, please cite:

- David Sehnal, Sebastian Bittrich, Mandar Deshpande, Radka Svobodová, Karel Berka, 
Václav Bazgier, Sameer Velankar, Stephen K Burley, Jaroslav Koča, Alexander S Rose: 
Mol* Viewer: modern web app for 3D visualization and analysis of large biomolecular structures, 
Nucleic Acids Research, 2021; https://doi.org/10.1093/nar/gkab314.

The `PDBeMolStar` is based on a panel `ReactiveHTML` pane and thus can be used as any pane in a web application or in Jupyter notebooks. 

Two CSS themes are available, to select them use `theme='default'` or `theme='dark'`.  Note that currently the theme setting is shared between all Mol* viewer instances, when executing the cell below the .css theme of the viewer above updates

In [ ]:
from panel_chemistry.pane import PDBeMolStar
import param
import panel as pn
pn.extension(sizing_mode="stretch_width", template="fast")

## Basic Examples

Download and display the '1qyn' molecule from the protein database:


In [ ]:
PDBeMolStar(
    height=500,
    molecule_id='1qyn')

Using a local file, using Jupyter to serve the file. 
Make sure url of the file to show is correct relative to where Jupyter notebook was started. For the example below, start Jupyter Notebook in the panel-chemistry root dir

In [ ]:
PDBeMolStar(
    name='Local File',
    sizing_mode='stretch_width',
    height=500,
    custom_data = {
        'url': 'http://localhost:8888/files/examples/assets/1qyn.pdb',
        'format': 'pdb'
    },
)

## Tweaking the viewer appearance and coloring

In [ ]:
molstar = PDBeMolStar(
    height=500,
    hide_water=True,
    theme='dark',
    lighting='metallic',
    hide_expand_icon=True,
    hide_animation_icon=True,
    sequence_panel=False,
    highlight_color='#d1fa07',
    bg_color='#040724',
    molecule_id='1qyn')

molstar

More customization options can be found by looking at the parameters defined on `PDBeMolStar`

The method `color` can be used to select and color parts of the molecule. See [PDBe Mol* wiki - helper methods](https://github.com/molstar/pdbe-molstar/wiki/3.-Helper-Methods) for details.

In [ ]:

# Color a part of chain A yellow and color anything that wasnt selected blue (updates in the viewer above)
data = {
    'start_residue_number': 10,
    'end_residue_number': 125,
    'struct_asym_id': 'A',
    'color': {'r': 255, 'g': 215, 'b': 0},
    'focus': False,
    }

molstar.color([data], non_selected_color = {'r': 0, 'g': 87, 'b': 183})

In [ ]:
# Clear the colors / selection
molstar.clear_selection()

## Custom data and alphafold view

Custom data can be loaded by providing an URL through the `custom_data` dictionary. the format of the custom data needs to be specified as well.

In [ ]:
PDBeMolStar(
    height=500,
    custom_data = {
        'url': 'https://alphafold.ebi.ac.uk/files/AF-O15552-F1-model_v1.cif',
        'format': 'cif'
    },
    alphafold_view=True, 
)

## Interactivity

In [ ]:
molstar = PDBeMolStar(
    height=500,
    molecule_id='1qyn')

parameters = [
    'molecule_id',
    'hide_water', 
    'hide_polymer',
    'visual_style',
    'lighting',
]
controls = pn.Param(molstar, parameters=parameters, width=300, sizing_mode="fixed")

pn.Row(controls, molstar)

## Creating an app

In [ ]:
local_pdbe = PDBeMolStar(
    name='Local File',
    sizing_mode='stretch_width',
    height=500,
    custom_data = {
        'url': 'https://raw.githubusercontent.com/MarcSkovMadsen/panel-chemistry/main/examples/assets/1qyn.pdb',
        'format': 'pdb'
    },
)

parameters = [
    'hide_water', 
    'hide_polymer',
    'visual_style',
    'lighting',
    'spin'
]

class Controller(param.Parameterized):
    chain = param.Selector(default='A', objects=['A', 'B', 'C', 'D'])
    apply_colors = param.Action(lambda self: self._action_color())
    reset_colors = param.Action(lambda self: self._action_reset_color())
    residues = param.Tuple((20, 50))
    highlight = param.Action(lambda self: self._action_highlight())
    clear_highlight = param.Action(lambda self: self.pdbe.clear_highlight())
    reset = param.Action(lambda self: self._action_reset())
    
    def __init__(self, pdbe, **params):
        self.pdbe = pdbe
        super().__init__(**params)
        
    def _action_color(self):
        """Color one chain yellow, color everything else blue"""
        
        data = {
            'struct_asym_id': self.chain,
            'color': {'r': 255, 'g': 215, 'b': 0}
        }
        
        self.pdbe.color([data], non_selected_color={'r': 0, 'g': 87, 'b': 183})

    def _action_reset_color(self):
        self.pdbe.clear_selection()
    
    def _action_highlight(self):
        data = {
            'start_residue_number': self.residues[0],
            'end_residue_number': self.residues[1],
            'struct_asym_id': self.chain,
            'color': {'r': 255, 'g': 105, 'b': 180},
        }
        
        self.pdbe.highlight([data])

    def _action_reset(self):
        data = {
            'camera': True,
            'theme': True,  # reset theme doesnt work
            'highlightcolor': True,
            'selectColor': True
        }
        self.pdbe.reset(data)

ctrl = Controller(local_pdbe)
slider = pn.widgets.IntRangeSlider(name='Residues', start=15, end=150, value=(20, 50), step=1)
ctrl_widgets = pn.Param(ctrl, name="Controls", widgets={'residues': slider})

In [ ]:
local_pdbe.servable()

In [ ]:
ctrl_widgets.servable(area="sidebar")

In [ ]:
pn.Param(local_pdbe, parameters=parameters, name="Controls").servable(area="sidebar")

In [ ]:
pn.state.template.param.update(site="Panel Chemistry", site_url="./", title="PDBe MolStar Pane", header_background='#0066cc', accent_base_color='#ffcc00');

Serve the app via `panel serve PDBe_MolStar.ipynb` and check it out at http://localhost:5006/PDBe_MolStar.

If you add the flag `--autoreload` you will get automatic reloading when ever you save the file.

## Local Files

If you want to use local files Panel can help you. Assuming file `1qyn.pdb` is in the folder `my_assets_folder` folder you can change the above code to

```python
local_pdbe = PDBeMolStar(
    name='Local File',
    sizing_mode='stretch_width',
    height=500,
    custom_data = {
        'url': 'assets/1qyn.pdb',
        'format': 'pdb'
    },
)
```

and serve the app via `panel serve PDBe_MolStar.ipynb --static-dirs assets=my_assets_folder`